In [1]:
import pandas as pd
import eddb
eddb.load_feeds()

# Found "C:\Users\komin\AppData\Local\Temp\elite-tools\archive-v6-systems_populated.jsonl".
# Feeds.POPULATED_SYSTEMS records loaded:  20629
# Found "C:\Users\komin\AppData\Local\Temp\elite-tools\archive-v6-systems_populated.jsonl".
# Feeds.POPULATED_SYSTEMS records loaded:  20629
# Found "C:\Users\komin\AppData\Local\Temp\elite-tools\archive-v6-stations.jsonl".
# Feeds.STATIONS records loaded:  49461
# Found "C:\Users\komin\AppData\Local\Temp\elite-tools\archive-v6-factions.jsonl".
# Feeds.FACTIONS records loaded:  77096
# Found "C:\Users\komin\AppData\Local\Temp\elite-tools\archive-v6-factions.jsonl".
# Feeds.FACTIONS records loaded:  77096


In [2]:
faction = "The Order of Mobius"
origin = eddb.faction_home_system(faction)
core_systems = set(eddb.query_systems_by_faction(faction))
extended_systems = set([inf_s for core_s in core_systems for inf_s in eddb.query_nearby_systems(core_s, eddb.SYSTEM_INFLUENCE_RANGE)])

In [3]:
faction_report = eddb.system_faction_report(extended_systems, faction, origin).sort_values(by=['Priority', 'Name'], ascending=[True, True])
faction_report.to_clipboard()
faction_report

,Priority,Category,Name,Distance,Population,Primary Economy,States,Happiness,Security Level,Controlling Faction,Influence,Government,Allegiance,Player Factions Present
52,0,Insurgency,Dvorsi,14.5,65081959,High Tech,Boom,2,High,The Order of Mobius,38.4000,Cooperative,Independent,"The Guardians, The Order of Mobius"
92,0,Insurgency,Njiri,12.0,17186307,High Tech,Boom,2,Medium,The Order of Mobius,48.4000,Cooperative,Independent,"Black Ram, The Order of Mobius"
72,1,Control,Apathaam,6.0,3506390,Industrial,Boom,2,Medium,The Order of Mobius,57.4000,Cooperative,Independent,The Order of Mobius
38,1,Control,Asase Ya,24.6,10832961797,Agriculture,None,2,High,The Order of Mobius,48.4577,Cooperative,Independent,The Order of Mobius
150,1,Control,Azrael,0.0,4500000,Extraction,"Boom, Civil Liberty",2,Medium,The Order of Mobius,60.0400,Cooperative,Independent,The Order of Mobius
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
126,5,NPC,Tewi,23.2,310841,Refinery,Election,2,Medium,Tewi Guardians of Tradition,26.7732,Patronage,Empire,
31,5,NPC,Thanatos,31.0,450000,Refinery,Boom,2,Medium,Thanatos Inc,48.8000,Corporate,Empire,
123,5,NPC,Wandjin,23.8,114523,Extraction,Boom,2,Medium,HR 571 Emperor's Grace,43.8000,Patronage,Empire,
11,5,NPC,Windjabar,51.1,3606428,Industrial,None,2,Medium,Kokonjekodi Empire Pact,33.1000,Patronage,Empire,


In [4]:
faction_stats = {}
#cpf = eddb.player_faction_controlled(extended_systems, faction, origin)
for s_name in extended_systems:
    s = eddb.populated_systems[s_name]
    cf_name = s['controlling_minor_faction']
    if cf_name in eddb.player_faction_names:
        if cf_name not in faction_stats.keys():
            faction_stats[cf_name] = {'name': cf_name, 'control': 0, 'presence': 0}
        faction_stats[cf_name]['control'] += 1
    for f in s['minor_faction_presences']:
        f_name = eddb.faction_names_by_id[f['minor_faction_id']]
        if f_name in eddb.player_faction_names:
            if f_name not in faction_stats.keys():
                faction_stats[f_name] = {'name': f_name, 'control': 0, 'presence': 0}
            if f_name != cf_name:
                faction_stats[f_name]['presence'] += 1

pd.DataFrame.from_dict(faction_stats.values()).set_index('name').sort_values(by=['control', 'presence', 'name'], ascending=[False, False, True])

,control,presence
name,,
The Order of Mobius,21,1
Lavigny's Legion,14,1
Decimus Imperium Lex,13,0
Silver Hunter Order,8,3
The 10th Legion,7,1
Daily Routine Incorporated,7,0
Libertas Co-operative,6,3
The White Templars,5,0
Empire Corsairs,4,4
